In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
import torch
import torch.utils.data
import torchvision
import torchvision.ops 
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from PIL import Image
from torchvision.transforms import functional as func
import cv2
import numpy as np
from detectron2.evaluation.coco_evaluation import instances_to_coco_json
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog
import pickle
from fiftyone import ViewField as F

/home/dongheehan/anaconda3/envs/objectdetection/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

inferencemode = 'COCO-Detection'
expendName = '.yaml'
load_model_name1 = 'retinanet_R_50_FPN_1x'
load_model_name = 'ensemble_union'

# load model
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file(os.path.join(inferencemode, load_model_name1 + expendName)))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
#Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(os.path.join(inferencemode, load_model_name1 + expendName))
predictor1 = DefaultPredictor(cfg)
predictor1.model.eval()

PREDICTION_PATH = './predictions'

# load
with open(os.path.join(PREDICTION_PATH, f'{load_model_name}_predictions.pickle'), 'rb') as f:
    model1_predictions = pickle.load(f)

print("Model ready")

Loading config /home/dongheehan/anaconda3/envs/objectdetection/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[Checkpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/retinanet_R_50_FPN_1x/190397773/model_final_bfca0b.pkl ...
URL https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/retinanet_R_50_FPN_1x/190397773/model_final_bfca0b.pkl cached in /home/dongheehan/.torch/iopath_cache/detectron2/COCO-Detection/retinanet_R_50_FPN_1x/190397773/model_final_bfca0b.pkl
Reading a file from 'Detectron2 Model Zoo'
The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
Model ready


In [3]:
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    dataset_name="evaluate-ensenble-detections",
)

dataset.persistent = True

# Print some information about the dataset
print(dataset)

# Print a ground truth detection
sample = dataset.first()
print(sample.ground_truth.detections[0])

Found annotations at '/home/dongheehan/fiftyone/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'evaluate-ensenble-detections'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Name:        evaluate-ensenble-detections
Media type:  image
Num samples: 5000
Persistent:  True
Tags:        ['validation']
Sample fields:
    id:                            fiftyone.core.fields.ObjectIdField
    filepath:                      fiftyone.core.fields.StringField
    tags:                          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth:                  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    ensemble:                      fiftyone.core.fields.EmbeddedDocumentFi

In [ ]:
session = fo.launch_app()

In [ ]:

predictions_view = dataset.take(5000,seed=51) # limit size

# Add predictions to samples
predictions= []
with fo.ProgressBar() as pb:
    for sample in pb(predictions_view):

        # Load image
        image = cv2.imread(sample.filepath)
        h, w, c = image.shape

        filename = os.path.basename(sample.filepath)

        detections = model1_predictions[filename]

        sample[load_model_name] = fo.Detections(detections=detections)
        sample.save()

print("Finished adding predictions")
session.view = None
session.view = predictions_view
session.show()

In [ ]:
# Only contains detections with confidence >= 0.75
high_conf_view = dataset.filter_labels(load_model_name, F("confidence") > 0.15)

print(high_conf_view)

sample = high_conf_view.first()
print(sample.ensemble.detections[0])

session.view = high_conf_view

In [7]:
eval_key = load_model_name

results = high_conf_view.evaluate_detections(
    load_model_name,
    gt_field="ground_truth",
    eval_key=eval_key,
    compute_mAP=True,
)



Evaluating detections...
 100% |███████████████| 4995/4995 [18.7m elapsed, 0s remaining, 4.1 samples/s]      
Performing IoU sweep...
 100% |███████████████| 4995/4995 [7.6m elapsed, 0s remaining, 11.2 samples/s]      


In [8]:
print(dataset.get_evaluation_info(eval_key))

{
    "key": "ensemble_union",
    "version": "0.16.2",
    "timestamp": "2022-06-11T21:39:20.836000",
    "config": {
        "method": "coco",
        "cls": "fiftyone.utils.eval.coco.COCOEvaluationConfig",
        "pred_field": "ensemble_union",
        "gt_field": "ground_truth",
        "iou": 0.5,
        "classwise": true,
        "iscrowd": "iscrowd",
        "use_masks": false,
        "use_boxes": false,
        "tolerance": null,
        "compute_mAP": true,
        "iou_threshs": [
            0.5,
            0.55,
            0.6,
            0.65,
            0.7,
            0.75,
            0.8,
            0.85,
            0.9,
            0.95
        ],
        "max_preds": 100,
        "error_level": 1
    }
}


In [9]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("ground_truth.detections.label")
classes_top = sorted(counts, key=counts.get, reverse=True)

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top)

                precision    recall  f1-score   support

        person       0.34      0.97      0.51     34218
           car       0.14      0.93      0.25      3500
         chair       0.11      0.85      0.19      2831
          book       0.33      0.96      0.49      6657
        bottle       0.17      0.92      0.29      2312
           cup       0.10      0.87      0.18      1121
  dining table       0.05      0.73      0.09       748
 traffic light       0.07      0.79      0.13       712
          bowl       0.08      0.86      0.15       748
       handbag       0.04      0.70      0.08       540
          bird       0.32      0.94      0.48      1887
          boat       0.15      0.92      0.26      1014
         truck       0.06      0.88      0.11       427
      umbrella       0.18      0.90      0.29       757
         bench       0.06      0.77      0.12       610
           cow       0.29      0.96      0.44      1042
        banana       0.27      0.94      0.42  

In [10]:
print(results.mAP())

0.2728808887914399
